In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torchtext.data as ttd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from datetime import datetime
from pathlib import Path
import pandas as pd

In [ ]:
folder=Path('/content/gdrive/My Drive/Teaching/NLP/8_Sequence_Code')

In [ ]:
# Generate some data 
data = {
    "label": ['negative', 'positive', 'positive'],
    "data": [
        "I like cakes and desert.",
        "Cakes I like!",
        "Cakes and desert or just desert?",
    ]
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

,label,data
0,negative,I like cakes and desert.
1,positive,Cakes I like!
2,positive,Cakes and desert or just desert?


In [ ]:
df.to_csv(folder / 'toydata.csv', index=False)

In [ ]:
!head '/content/gdrive/My Drive/Teaching/NLP/8_Sequence_Code/toydata.csv'

label,data
negative,I like cakes and desert.
positive,Cakes I like!
positive,Cakes and desert or just desert?


In [ ]:
TEXT = ttd.Field(
    sequential=True,
    batch_first=True,
    lower=True,
    tokenize='spacy',
    pad_first=True)

LABEL = ttd.LabelField(dtype = torch.float)


dataset = ttd.TabularDataset(
    path=folder / 'toydata.csv',
    format='csv',
    skip_header=True,
    fields=[('label', LABEL), ('data', TEXT)]
)

In [ ]:
ex = dataset.examples[0]

In [ ]:
type(ex)

torchtext.data.example.Example

In [ ]:
ex.data

['i', 'like', 'cakes', 'and', 'desert', '.']

In [ ]:
ex.label

'negative'

In [ ]:
train_dataset, test_dataset = dataset.split(0.8) # default is 0.7

In [ ]:
TEXT.build_vocab(train_dataset)
LABEL.build_vocab(train_dataset)

In [ ]:
vocab_text = TEXT.vocab
type(vocab_text)

torchtext.vocab.Vocab

In [ ]:
vocab_text.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'.': 5,
             '<pad>': 1,
             '<unk>': 0,
             '?': 6,
             'and': 3,
             'cakes': 4,
             'desert': 2,
             'i': 7,
             'just': 8,
             'like': 9,
             'or': 10})

In [ ]:
vocab_text.itos

['<unk>',
 '<pad>',
 'desert',
 'and',
 'cakes',
 '.',
 '?',
 'i',
 'just',
 'like',
 'or']

In [ ]:
vocab_label = LABEL.vocab
type(vocab_label)

torchtext.vocab.Vocab

In [ ]:
vocab_label.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'negative': 0, 'positive': 1})

In [ ]:
vocab_label.itos

['negative', 'positive']

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# by default train data is shuffled for each epoch, test and tarin are not
# we sort by data aatribute to minimize padding
train_iter, test_iter = ttd.BucketIterator.splits((train_dataset,test_dataset), 
                              sort_key=lambda x: len(x.data),
                              batch_sizes=(2,2), 
                              device=device)

In [ ]:
for batch in train_iter:
  print("inputs:", batch.data, "shape:", batch.data.shape)
  print("targets:",batch.label, "shape:", batch.label.shape)
  break

inputs: tensor([[ 1,  7,  9,  4,  3,  2,  5],
        [ 4,  3,  2, 10,  8,  2,  6]], device='cuda:0') shape: torch.Size([2, 7])
targets: tensor([0., 1.], device='cuda:0') shape: torch.Size([2])


In [ ]:
for batch in test_iter:
  print("inputs:", batch.data, "shape:", batch.data.shape)
  print("targets:",batch.label, "shape:", batch.label.shape)
  break

inputs: tensor([[4, 7, 9, 0]], device='cuda:0') shape: torch.Size([1, 4])
targets: tensor([1.], device='cuda:0') shape: torch.Size([1])
